In [2]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from itertools import combinations
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import contractions
from tqdm import tqdm

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

# Read the JSON Lines file
with open('ML Datasets/ML Dataset 1/News_Category_Dataset_v3.json', 'r') as infile:
    lines = infile.readlines()

# Add commas between JSON objects and format as a list
with open('output.json', 'w') as outfile:
    outfile.write('[\n')  # Start of JSON array
    for i, line in enumerate(lines):
        # Strip newline characters and trailing whitespace
        line = line.strip()
        # Add comma at the end of each line except the last one
        if i < len(lines) - 1:
            outfile.write(line + ',\n')
        else:
            outfile.write(line + '\n')
    outfile.write(']\n')  # End of JSON array


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to

In [7]:
df = pd.read_json("output.json")
print(df.head())
print(df.shape)

                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog y

In [8]:
df = df[df['link'] != ""]
df = df[df['headline'] != ""]
df = df[df['category'] != ""]
print(df.shape)

df_baljyot = df.iloc[30000:,:]
df.shape

(209521, 6)


(209521, 6)

In [10]:
import concurrent.futures

# Applying tqdm to panda dataframes to observe processing progress for large datasets
tqdm.pandas()

# Function to get the text of the article from a given URL
def fetch_article_text(url):
    try:
        # Sending a request to the URL
        response = requests.get(url, timeout = 10)
        
        # Checking to see if the request was successful
        if response.status_code == 200:
            # Parsing the HTML content
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extracting the main article text
            article_text = ' '.join([p.get_text() for p in soup.find_all('p')])
            return article_text
        else:
            print(f"Request failed: {url}")
            return None
    except Exception as e:
        print(f"Exception: Error fetching article from {url}: {e}")
        return None

# Parallel function to fetch article texts for a given DataFrame chunk
def fetch_article_chunk(index_url_pair):
    idx, url = index_url_pair
    return (idx, fetch_article_text(url))

# Fetching the article's text for each link in parallel
df_from_csv = pd.read_csv('ML Datasets/ML Dataset 1/debjit.csv')

# Prepare a placeholder NumPy array to store the article texts in order
article_texts = np.empty(len(df_baljyot), dtype=object)


In [12]:
# Prepare the index-url pairs to pass to the executor (maintains original index)
index_url_pairs = list(df_baljyot.iloc[:, 0].items())  # (index, URL)

# Function to process a chunk of URLs
def process_chunk(start_idx, end_idx):
    chunk = index_url_pairs[start_idx:end_idx]
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map the function to the index-url pairs for the current chunk
        futures = list(tqdm(executor.map(fetch_article_chunk, chunk), total=len(chunk)))
    
    # Assign the article texts to the correct index positions
    for idx, text in futures:
        article_texts[idx] = text

process_chunk(0, 2500)
# process_chunk(2500, 5000)
# process_chunk(5000, 7500)
# process_chunk(7500, 10000)
# process_chunk(10000, 12500)
# process_chunk(12500, len(df_from_csv))

 12%|█▏        | 312/2500 [00:40<06:24,  5.70it/s]

Request failed: https://www.huffingtonpost.com/entry/exploring-the-gay-straight-divide_us_5965263ce4b09be68c0055ca


 17%|█▋        | 428/2500 [00:57<03:22, 10.21it/s]

Exception: Error fetching article from https://www.huffingtonpost.comhttps://www.theguardian.com/lifeandstyle/2017/jul/10/workin-it-how-female-drag-queens-are-causing-a-scene: HTTPSConnectionPool(host='www.huffingtonpost.comhttps', port=443): Max retries exceeded with url: /www.theguardian.com/lifeandstyle/2017/jul/10/workin-it-how-female-drag-queens-are-causing-a-scene (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000208AEA085F0>: Failed to resolve 'www.huffingtonpost.comhttps' ([Errno 11001] getaddrinfo failed)"))


 22%|██▏       | 556/2500 [01:14<04:19,  7.49it/s]


KeyboardInterrupt: 

In [35]:
# Adding the fetched article texts back to the DataFrame
df_baljyot['article_text'] = article_texts

# Saving the modified DataFrame to a new CSV
df_baljyot.to_csv('ML Datasets/ML Dataset 1/baljyot_mod.csv', index=False)

# df_from_csv.head()
df_from_csv.iloc[0,0]
# print(article_texts)

'https://www.huffingtonpost.com/entry/marni-for-hm_us_5b9b6d8ee4b03a1dcc77773f'

In [36]:
df_from_csv.iloc[0,df_from_csv.shape[1]-1]

'Author, travel writer and mom, lover of la dolce vita Stop the presses. I am officially a door buster.  I will go to great lengths for style. And, am often mortified (post acquisition) at what I will/would/will endure to hunt and gather the objects of my affection. Case in point? The activities surrounding the March 8 sale of uber expensive Marni for H&M. Now, I have been bowing down to the temple of Marni since the early 90\'s. Designer Consuelo Castiglione\'s upbeat prints and signature slouchy fit are my version of wardrobe perfection. But, in order to score discount designer duds, one must wade through the degradation that punctuates a fast fashion shopping experience. Having spent decades conquering sample sales in NYC and LA, I knew that my chances at nabbing my wish list would double if I had a partner. So, I enlisted my trusty assistant, Alaina, to coordinate an attack plan. In order to snap up fabulous fast fashion, one must physically go to the store. Very early. You see, th

In [66]:
# Function to remove the punctuation marks from the given text
def remove_punctuation(text):

    # Returning the cleaned text for further processing
    return re.sub(r'[^\w\s]', '', text)

# Function to remove the stopwords from the given text
def remove_stopwords(text):

    # Defining a set of English stopwords
    stop_words = set(stopwords.words('english'))

    # Tokenizing the given text
    tokens = word_tokenize(text)

    # Removing the stop words from the given text based on the above created English stopword set
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Combining tokens back into a single string
    cleaned_text = ' '.join(filtered_tokens)

    # Returning the cleaned text for further processing
    return cleaned_text

# Function to lemmatize the given text
def lemmatize_text(text):

    # Initializing the lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Tokenizing the given text
    tokens = word_tokenize(text)
    
    # Lemmatizing each token of the given text to break them down to their root forms
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Combining the lemmatized tokens back into a single string abd returning it for further processing
    return ' '.join(lemmatized_tokens)

# Function to remove URLs from the given text
def remove_urls(text):

    # Defining the URL pattern to remove from the given text
    url_pattern = r'(?:(?:http|https|ftp|ftps|sftp|file|mailto|tel|ws|wss)://|www\.)[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)+[^\s]*'

    # Returning the new text with all URLs removed for further processing
    return re.sub(url_pattern, '', text)

# Function to remove the html tags (if any) from the given text
def remove_html_tags(text):

    # Creating a BeautifulSoup object and parsing the text with html.parser
    soup = BeautifulSoup(text, "html.parser")

    # Getting the text without HTML tags and returning it for further processing
    return soup.get_text()

# Function to expand contractions (for example: "don't" to "do not")
def expand_contractions(text):

    # Returning the new text with expanded words for further processing
    return contractions.fix(text)

# Function to remove the numbers from the given text
def remove_numbers(text):
    
    # Returning the new text with numbers removed for further processing
    return re.sub(r'\d+', '', text)

# Function to keep only valid English words in the given text
def remove_invalid_words(text):

    # Tokenizing the given text
    tokens = word_tokenize(text)

    # # Get the list of valid English words
    word_list = set(words.words())

    # Checking validity for each word
    valid_words = [word for word in tokens if word in word_list]

    # Returning the list of valid English words extracted from the given text for further processing
    return ' '.join(valid_words)

#  Preprocessing function for cleaning the article's text for further processing
def preprocess_text(text, to_remove_stopwords = True, lemmatize = True):

    # Converting all the article's text to lowercase
    text = text.lower()

    # Removing all the urls from the article's text (keeping this before removing punctuation to avoid problems)
    text = remove_urls(text)

    # Removing all the punctuation marks from the article's text
    text = remove_punctuation(text)

    text = remove_html_tags(text) # Removing the html tags (if any)
    text = expand_contractions(text) # Expanding contractions (For example: "don't" to "do not")
    text = remove_numbers(text) # Removing numbers from the text

    text = remove_invalid_words(text)

    # Removing stop words
    if(to_remove_stopwords):
        text = remove_stopwords(text)
    
    # Lemmatizing the words to break them to their root meanings
    if(lemmatize):
        text = lemmatize_text(text)    

    # Returning the final clean text
    return text

# Test Example for the article text preprocessing function
# text = "Hi! My name is abc. What is your name? my number is 123, https://www.youtube.com/ dlfjk kdkd dkdkdkd  <break> don't hadn't won't  "
# text2 = "using https://www.google.com/ as an example"

# ptext = preprocess_text(text)
# ptext2 = preprocess_text(text2)
# print(ptext)
# print(ptext2)

# Main implementation for the rows of the article dataframe
df['cleaned_text'] = df['article_text'].progress_apply(preprocess_text)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [04:29<00:00,  3.71it/s]


In [4]:
df1=pd.read_csv('ML Datasets/Ml Dataset 1/debjit_mod.csv')
df2=pd.read_csv('ML Datasets/Ml Dataset 1/vijval_mod.csv')
df3=pd.read_csv('ML Datasets/Ml Dataset 1/jaleel_mod.csv')
df4=pd.read_csv('ML Datasets/Ml Dataset 1/baljyot_mod.csv')
df_combined = pd.concat([df1, df2,df3, df4], ignore_index=True)

df_combined.to_csv("ML Datasets/Ml Dataset 1/combined_extracted.csv")

In [3]:
df=pd.read_csv('ML Datasets/ML Dataset 1/combined_without_nans.csv')
df["weighted_text"]= 10*(df['cleaned_headline']+" ")+ 4*(df['cleaned_short_description']+" ")+df["cleaned_article_text"]

In [4]:
### OkapiBM25

# # Preprocess the entire corpus
# cleaned_texts = [preprocess_text(doc) for doc in df.iloc[:,-1]]

# # Tokenize the cleaned corpus
# tokenized_texts = [word_tokenize(doc) for doc in cleaned_texts]

# # Create a BM25 model
# bm25_model = BM25Okapi(tokenized_texts)

# # Create vocabulary from the entire corpus
# vocabulary = list(set(word for doc in tokenized_texts for word in doc))

# # Create a BM25 score matrix
# bm25_score_matrix = []

# for word in vocabulary:
#     # Get BM25 scores for each document for the current word
#     score = bm25_model.get_scores(word)
#     bm25_score_matrix.append(score)

# # Convert the BM25 score matrix to a DataFrame and transpose to get a 2x5 shape
# bm25_features = pd.DataFrame(np.array(bm25_score_matrix).T, columns=vocabulary)

# bm25_features_df = bm25_features

# bm25_features_df['id'] = [i for i in range(1000)]
# print(bm25_features_df)

### TF-IDF Feature Extraction

# Initialize the TF-IDF vectorizer
vectorizer_tf = TfidfVectorizer(max_features=5000) 

# Fit and transform the cleaned text, with a progress bar
# Using a custom approach since TfidfVectorizer does not support tqdm directly
tfidf_matrix = vectorizer_tf.fit_transform(df['weighted_text'])

# Convert the TF-IDF matrix to a DataFrame for better visualization (optional)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer_tf.get_feature_names_out())

# Display the first few rows of the TF-IDF DataFrame
tfidf_df.head()


,aaron,abandoned,abc,ability,able,abortion,abroad,absence,absolute,absolutely,...,young,younger,youngest,youth,youtube,zealand,zero,zika,zimmerman,zone
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.037671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01715,0.024961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
### LDA

# Sample preprocessed text data
documents =list(df['weighted_text'])

# Step 1: Create a document-term matrix (Bag of Words)
vectorizer = CountVectorizer()
doc_term_matrix = vectorizer.fit_transform(documents)

# Step 2: Apply LDA
lda_model = LatentDirichletAllocation(n_components=15, random_state=42)
lda_features = lda_model.fit(doc_term_matrix)

lda_features_df = pd.DataFrame(lda_features, columns=[f'Component {i}' for i in range(lda_model.n_components)])

print(lda_features_df)

KeyboardInterrupt: 

In [72]:
# Define the number of topics for NMF
n_topics = 100

# Initialize NMF model
nmf_model = NMF(n_components=n_topics, random_state=42)

# Fit the NMF model to the combined matrix
nmf_features = nmf_model.fit_transform(doc_term_matrix)

# Create a DataFrame for the NMF features
nmf_features_df = pd.DataFrame(nmf_features, columns=[f'Component {i}' for i in range(n_topics)])

print(nmf_features_df)

     Component 0  Component 1  Component 2  Component 3  Component 4  \
0       0.000000          0.0          0.0          0.0          0.0   
1       0.006889          0.0          0.0          0.0          0.0   
2       0.013777          0.0          0.0          0.0          0.0   
3       0.000000          0.0          0.0          0.0          0.0   
4       0.027555          0.0          0.0          0.0          0.0   
..           ...          ...          ...          ...          ...   
995     6.854252          0.0          0.0          0.0          0.0   
996     6.858834          0.0          0.0          0.0          0.0   
997     6.868029          0.0          0.0          0.0          0.0   
998     6.874918          0.0          0.0          0.0          0.0   
999     6.881807          0.0          0.0          0.0          0.0   

     Component 5  Component 6  Component 7  Component 8  Component 9  ...  \
0            0.0          0.0          0.0          0.0   

In [73]:
### Merging the POS Tagging, TF-IDF, and LDA featurs into a single dataframe

# pos_and_tfidf_features_df = pd.DataFrame()

# # Merging POS tagging features with the TF-IDF features
# for index, row in tfidf_features_df.iterrows():
#     # print(index)
#     for i in range(0,len(row)-1):
#         word = tfidf_features_df.columns[i]
#         # print(word)
#         row[i] = {
#             'tfidf_score': row[i],
#             'pos_tag': pos_tagging_features[index].get(word, 'NULL')
#         }
#         # print(row[i])
#     # print(row)
#     pos_and_tfidf_features_df = pos_and_tfidf_features_df._append(row,ignore_index = True)

# print(pos_and_tfidf_features_df)

# Merging the LDA features with above combined dataframe
final_features_df = pd.merge(bm25_features_df, lda_features_df, on='id', how='left')

# Displaying the combined features DataFrame
print(final_features_df.head())

# Merging the LDA features with above combined dataframe
final_features_df_2 = pd.merge(bm25_features_df, nmf_features_df, on='id', how='left')

# Displaying the combined features DataFrame
print(final_features_df_2.head())

   heartiness  markedly  scare  brilliant  spiraled  moppy  contradiction  \
0         0.0       0.0    0.0        0.0       0.0    0.0            0.0   
1         0.0       0.0    0.0        0.0       0.0    0.0            0.0   
2         0.0       0.0    0.0        0.0       0.0    0.0            0.0   
3         0.0       0.0    0.0        0.0       0.0    0.0            0.0   
4         0.0       0.0    0.0        0.0       0.0    0.0            0.0   

   map  retired  need  ...  Component 90  Component 91  Component 92  \
0  0.0      0.0   0.0  ...      0.010000      0.010000      0.010000   
1  0.0      0.0   0.0  ...      0.005000      0.005000      0.005000   
2  0.0      0.0   0.0  ...      0.003333      0.003333      0.003333   
3  0.0      0.0   0.0  ...      0.000003      0.000003      0.000003   
4  0.0      0.0   0.0  ...      0.002000      0.002000      0.002000   

   Component 93  Component 94  Component 95  Component 96  Component 97  \
0      0.010000      0.010000

In [74]:
## Adding the required columns from the original dataframe for creating a proper dataframe with labels

nmf_features_df['headline'] = df['headline']
nmf_features_df['date'] = df['date']
nmf_features_df['category'] = df['category']

nmf_features_df.head()

,Component 0,Component 1,Component 2,Component 3,Component 4,Component 5,Component 6,Component 7,Component 8,Component 9,...,Component 94,Component 95,Component 96,Component 97,Component 98,Component 99,id,headline,date,category
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0,Over 4 Million Americans Roll Up Sleeves For O...,2022-09-23,U.S. NEWS
1,0.006889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,1,"American Airlines Flyer Charged, Banned For Li...",2022-09-23,U.S. NEWS
2,0.013777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,2,23 Of The Funniest Tweets About Cats And Dogs ...,2022-09-23,COMEDY
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030877,...,0.0,0.0,0.0,0.0,0.046971,0.0,3,The Funniest Tweets From Parents This Week (Se...,2022-09-23,PARENTING
4,0.027555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,4,Woman Who Called Cops On Black Bird-Watcher Lo...,2022-09-22,U.S. NEWS


## Randomly Generated Dataset for User Interaction Data

In [24]:
import random

user_df = pd.DataFrame(columns = [f'Article {i}' for i in range(df.shape[0])])

for i in range(50):
    temp_dict = {'user_id': i}
    for j in range(df.shape[0]):
        clicks = random.randint(0,25)
        like = random.choice([0,1]) if clicks >= 5 else 0
        review = random.choice([0,1]) if clicks >= 10 else 0
        temp_dict[f'Article {j}'] = clicks*0.01 + like*0.25 + review*0.5
    user_df = user_df._append(temp_dict, ignore_index = True)

user_df.head()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_24128\1639066178.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  user_df = user_df._append(temp_dict, ignore_index = True)


,Article 0,Article 1,user_id
0,0.73,0.41,0.0
1,0.35,0.72,1.0
2,0.15,0.33,2.0
3,0.19,0.01,3.0
4,0.03,0.04,4.0


## Method to Combine Collaborative and Content Based Filtering methods for Final Prrediction

In this we have to use the user interaction data for collaborative based filtering and the text article data for content based filtering.

1) For less active users, we can use only content based filtering using a clustering model, and for which we can use the user interaction data for figuring out which articles the user has read, using a small threshold like 0.1 or 0.25.

2) For more active users with a lot of interaction history (a user has to have more than 1000 or 5000 articles with scores >= 0.5), we can use a weighted result of both content based filtering and collaborative based filtering models.

In [ ]:
import numpy as np
import skfuzzy as fuzz
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

cleaned_and_extracted_df = pd.read_csv("ML Datasets/ML Dataset 1/combined_cleaned_and_extracted.csv")

# Number of clusters
n_clusters = 15

# Apply Fuzzy C-Means
centres, initial_membership_matrix, final_membership_matrix, distances_matrix, obj_func_values, iteration_count, fpc = fuzz.cluster.cmeans(
    cleaned_and_extracted_df, n_clusters, 2, error=1e-5, maxiter=100, init=None)

# Get the hard cluster membership for each data point
cluster_membership = np.argmax(final_membership_matrix, axis=0)

# Apply t-SNE to reduce to 3D
tsne = TSNE(n_components=3, random_state=42)
data_3d = tsne.fit_transform(cleaned_and_extracted_df.T)  # Transpose to match dimensions

# Plotting the clusters in 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot each cluster with a different color
for j in range(n_clusters):
    ax.scatter(data_3d[cluster_membership == j, 0],
               data_3d[cluster_membership == j, 1],
               data_3d[cluster_membership == j, 2],
               label=f'Cluster {j + 1}', s=50)

# Plot the cluster centers
center_3d = tsne.fit_transform(centres)  # Cluster centers in 3D
ax.scatter(center_3d[:, 0], center_3d[:, 1], center_3d[:, 2],
           s=300, marker='X', c='black', label='Centers')

# Customize plot
ax.set_title('3D t-SNE of Fuzzy C-Means Clustering')
ax.set_xlabel('t-SNE Dim 1')
ax.set_ylabel('t-SNE Dim 2')
ax.set_zlabel('t-SNE Dim 3')
ax.legend()
plt.show()


TypeError: can't multiply sequence by non-int of type 'float'